In [2]:
import pandas as pd
import numpy as np
movies = pd.read_csv("//kaggle/input/tmdb-movies-daily-updates/TMDB_all_movies.csv")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089080 entries, 0 to 1089079
Data columns (total 28 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1089080 non-null  int64  
 1   title                    1089068 non-null  object 
 2   vote_average             1089080 non-null  float64
 3   vote_count               1089080 non-null  float64
 4   status                   1089080 non-null  object 
 5   release_date             976976 non-null   object 
 6   revenue                  1089080 non-null  float64
 7   runtime                  1089080 non-null  float64
 8   budget                   1089080 non-null  float64
 9   imdb_id                  624365 non-null   object 
 10  original_language        1089080 non-null  object 
 11  original_title           1089068 non-null  object 
 12  overview                 918797 non-null   object 
 13  popularity               1089080 non-null 

In [3]:
columns = [
    'id',
    'title',
    'overview',
    'original_language',
    'genres',
    'cast',
    'director',
    'writers',
    'release_date',
    'imdb_rating',
    'poster_path'
]

movies = movies[columns]
movies = movies[movies['original_language'].isin(['en', 'hi'])]

In [4]:
movies.info()


<class 'pandas.core.frame.DataFrame'>
Index: 549838 entries, 2 to 1089077
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 549838 non-null  int64  
 1   title              549827 non-null  object 
 2   overview           537108 non-null  object 
 3   original_language  549838 non-null  object 
 4   genres             373055 non-null  object 
 5   cast               355463 non-null  object 
 6   director           436212 non-null  object 
 7   writers            245383 non-null  object 
 8   release_date       478523 non-null  object 
 9   imdb_rating        236438 non-null  float64
 10  poster_path        389483 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 50.3+ MB


In [5]:
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies['original_language'] = movies['original_language'].astype('category')
#movies = movies[movies['release_date']<='2024-12-31']

In [6]:
movies['original_language'][movies['original_language'].str.startswith('te')].unique()

[], Categories (2, object): ['en', 'hi']

In [7]:
movies.shape

(549838, 11)

In [8]:
movies['tags'] = movies['overview']+movies['genres']+movies['cast']+movies['director']+movies['writers']

In [9]:
train_df = movies[['id','title','tags']]
movies_df = movies[['id','title','imdb_rating','overview','release_date','poster_path']]

In [46]:
train_df.shape

(197178, 3)

In [11]:
train_df = train_df.dropna()
movies_df = movies_df.dropna()

In [12]:
train_df['tags'] = train_df['tags'].apply(lambda x: x.lower())

In [13]:
train_df.head()

,id,title,tags
2,5,Four Rooms,it's ted the bellhop's first night on the job....
3,6,Judgment Night,"four young friends, while taking a shortcut en..."
6,11,Star Wars,princess leia is captured and held hostage by ...
7,12,Finding Nemo,"nemo, an adventurous young clownfish, is unexp..."
8,13,Forrest Gump,a man with a low iq has accomplished great thi...


In [14]:
#pip install faiss-cpu

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')
vectors = cv.fit_transform(train_df['tags']).toarray().astype('float32')

In [21]:
import faiss
faiss.normalize_L2(vectors)

In [22]:
index = faiss.IndexFlatIP(vectors.shape[1])
index.add(vectors)


In [23]:
id_to_title = train_df['title'].tolist()

In [24]:
'''from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # 6 includes the movie itself
model.fit(vectors)'''


"from sklearn.neighbors import NearestNeighbors\n\nmodel = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # 6 includes the movie itself\nmodel.fit(vectors)"

In [56]:
train_df = train_df.reset_index(drop=True)
movies_df = movies_df.reset_index(drop=True)

In [57]:
def recommend(movie_title, top_n=5):
    matches = train_df[train_df['title'] == movie_title]
    
    if matches.empty:
        print(f"❌ Movie '{movie_title}' not found in the dataset.")
        return
    
    movie_idx = matches.index[0]
    
    # Perform FAISS search
    distances, indices = index.search(vectors[movie_idx].reshape(1, -1), k=top_n+1)
    
    print(f"\nTop {top_n} similar movies to '{movie_title}':")
    for i, score in zip(indices[0][1:], distances[0][1:]):
        print(id_to_title[i])

recommend("Deadpool 2")


Top 5 similar movies to 'Deadpool 2':
Once Upon a Deadpool
Deadpool & Wolverine
THERE
Deadpool
The Lake in the Sky


In [50]:
faiss.write_index(index, "movie_index.faiss")


In [52]:
import pickle

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(cv, f)


In [58]:
movies_df.to_csv("movies_df.csv", index=False)


In [59]:
df1 = pd.read_csv('/kaggle/working/movies_df.csv')

In [60]:
df1.shape

(194990, 6)